In [1]:
import tweepy
import re
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import pickle
import twitterkey.key as access_key

with open("logi_model.pkl",'rb') as mod:
    model=pickle.load(mod)
with open("vector.pkl",'rb') as vec:
    vector=pickle.load(vec)
    
key=access_key.validation()

def sentiment(model,vector):
    consumer_key = key[0] 
    consumer_secret = key[1] 
    access_token_key = key[2] 
    access_token_secret = key[3] 
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token_key, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    # input for term to be searched and how many tweets to search
    searchTerm = input("\nEnter Keyword/Tag to search about: ")
    NoOfTerms = int(input("Enter how many tweets to search: "))
    # searching for tweets
    tweets = tweepy.Cursor(api.search, q=searchTerm, lang = "en").items(NoOfTerms)
        
    df=pd.DataFrame([tweet.text for tweet in tweets],columns=["tweet"])

    def remove_pattern(input_txt, pattern):
        r = re.findall(pattern, input_txt)
        for word in r:
            input_txt = re.sub(word, "", input_txt)
            input_txt=re.sub(r"RT[\s]+","",input_txt) #Remove Re-tweet
            input_txt=re.sub(r"https?:\/\/\S+","",input_txt)
        return input_txt

    df['clean_tweet'] = np.vectorize(remove_pattern)(df['tweet'], "@[\w]*")
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>2]))
    tokenized_tweet = df['clean_tweet'].apply(lambda x: x.split())
    tokenized_tweet.head()


    stemmer = PorterStemmer()

    tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
    tokenized_tweet.head()


    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = " ".join(tokenized_tweet[i])


    df['clean_tweet'] = tokenized_tweet

    x = vector.transform(df['clean_tweet'])
    
    predicted=model.predict(x)
    y=np.where(predicted==0,"negative","positve")
    return df["clean_tweet"],y


In [2]:
x,y=sentiment(model,vector)


Enter Keyword/Tag to search about: MiIndiaSupport
Enter how many tweets to search: 100


In [3]:
output_data=pd.DataFrame(data=[x,y]).T
output_data.columns=["Tweet","Sentiment"]

In [4]:
output_data

,Tweet,Sentiment
0,"karan, thi definit not the experi want you thr...",positve
1,when ultra will avail buy. it' been month sinc...,positve
2,"zaid, thank you for reach out us. thi definit ...",positve
3,understand your concern regard the replacement...,positve
4,despit msging and tweeting…,positve
...,...,...
95,the time slot and the servic centr address wil...,positve
96,"hello, measur precaut against covid-19 and owe...",positve
97,"aaditya, are sorri know that you are face issu...",negative
98,"hello ameeruddin, we'r sorri for the troubl yo...",positve


In [5]:
output_data["Sentiment"].value_counts()

positve     61
negative    39
Name: Sentiment, dtype: int64